In [ ]:

import matplotlib.pyplot as plt
import os

import drjit as dr
import mitsuba as mi

mi.set_variant('cuda_ad_rgb', 'llvm_ad_rgb')

sensor_count = 1
sensor = {
    'type': 'batch',
    'film': {
        'type': 'hdrfilm',
        'width': 256 * sensor_count, 'height': 256,
        'filter': {'type': 'gaussian'},
        'sample_border': True
    }
}

from mitsuba import ScalarTransform4f as T

golden_ratio = (1 + 5**0.5)/2
for i in range(sensor_count):
    theta = 2 * dr.pi * i / golden_ratio
    phi = dr.acos(1 - 2*(i+0.5)/sensor_count)

    d = 5
    origin = [
        d * dr.cos(theta) * dr.sin(phi),
        d * dr.sin(theta) * dr.sin(phi),
        d * dr.cos(phi)
    ]

    sensor[f"sensor_{i}"] = {
        'type': 'perspective',
        'fov': 45,
        'to_world': T.look_at(target=[0, 0, 0], origin=origin, up=[0, 1, 0])
    }

scene_dict = {
    'type': 'scene',
    'integrator': {
        'type': 'direct',
    },
    'sensor': sensor,
    'emitter': {
        'type': 'envmap',
        'filename': "C:/envmap2.exr",
    },
    'shape': {
        'type': 'ply',
        'filename': "C:/WorkingSets/3D/Clouds/generated_cloud_0.ply",
        'bsdf': {'type': 'diffuse'}
    }
}

scene_target = mi.load_dict(scene_dict)

def plot_batch_output(out: mi.TensorXf):
    fig, ax = plt.subplots(figsize=(5*sensor_count, 5))
    ax.imshow(mi.util.convert_to_bitmap(out))
    ax.axis('off')

ref_img = mi.render(scene_target, spp=256)
plot_batch_output(ref_img)


scene_dict['shape']['filename'] = 'C:/WorkingSets/3D/Clouds/point_cloud_0_scene.ply'
scene_source = mi.load_dict(scene_dict)

init_img = mi.render(scene_source, spp=128)
plot_batch_output(init_img)

